In [1]:
import requests
import string
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from gensim import corpora, models
from gensim.models import CoherenceModel
import numpy as np

## scraping berita kompas

In [2]:
web = requests.get("https://surabaya.kompas.com/read/2025/10/13/073714878/keharuan-keluarga-saat-identitas-2-jasad-santri-ponpes-al-khoziny")
soup = BeautifulSoup(web.content, 'html.parser')
for a in soup(['script', 'style']):
    a.decompose()
text = ' '.join(soup.stripped_strings)
print("Scraping berhasil. Teks mentah siap diproses.")

Scraping berhasil. Teks mentah siap diproses.


## inisisalisasi preprocessing

In [3]:
# Membuat stemmer dari Sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Mengunduh dan menyiapkan daftar stopwords Bahasa Indonesia
nltk.download('stopwords')
stop_words = set(stopwords.words('indonesian'))

print("Stemmer dan Stopwords siap digunakan.")

Stemmer dan Stopwords siap digunakan.


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## preprocessing per kalimat

In [4]:
# Memecah teks mentah menjadi daftar kalimat (sentences)
sentences = re.split(r'[.!?]\s*', text) # Memecah berdasarkan tanda titik, tanda tanya, atau tanda seru diikuti oleh spasi
print(f"Teks berhasil dipecah menjadi {len(sentences)} kalimat.\n") # Ditambah \n

#perulangan untuk preprocessing pada setiap kalimat 
processed_docs = []
for sentence in sentences:
    # Hanya proses kalimat yang tidak terlalu pendek
    if len(sentence) > 20:
        #Case Folding & Cleaning (angka, tanda baca)
        sent = sentence.lower()
        sent = re.sub(r'\d+', '', sent)
        sent = sent.translate(str.maketrans('', '', string.punctuation))
        sent = sent.strip()

        #Stemming
        stemmed_sent = stemmer.stem(sent)

        #Tokenizing & Stopword Removal
        tokens = [word for word in stemmed_sent.split() if word not in stop_words]

        # Tambahkan hasil (daftar token) jika tidak kosong
        if tokens:
            processed_docs.append(tokens)

print(f"Preprocessing selesai. Dihasilkan {len(processed_docs)} dokumen yang siap diproses.")
print("\nContoh 3 dokumen pertama:")
for i, doc in enumerate(processed_docs[:3]):
    print(f"Dokumen {i}: {doc}")

Teks berhasil dipecah menjadi 56 kalimat.

Preprocessing selesai. Dihasilkan 42 dokumen yang siap diproses.

Contoh 3 dokumen pertama:
Dokumen 0: ['haru', 'keluarga', 'identitas', 'jasad', 'santri', 'ponpes', 'al', 'khoziny', 'identifikasi', 'otomatis', 'mode', 'gelap', 'mode', 'terang', 'login', 'gabung', 'kompas']
Dokumen 1: ['com', 'konten', 'simpan', 'konten', 'suka', 'atur', 'minat', 'ikan', 'masuk', 'langgan', 'kompas', 'one', 'news', 'nasional', 'global', 'megapolitan', 'regional', 'milu', 'hype', 'konsultasi', 'hukum', 'cek', 'fakta', 'surat', 'baca', 'indeks', 'kilas', 'daerah', 'kilas', 'korporasi', 'kilas', 'menteri', 'sorot', 'politik', 'kilas', 'badan', 'negara', 'kelana', 'indonesia', 'kalbe', 'health', 'corner', 'kilas', 'parlemen', 'kilas', 'bumn', 'nusaraya', 'sumatera', 'utara', 'sumatera', 'selatan', 'sumatera', 'barat', 'riau', 'lampung', 'banten', 'yogyakarta', 'jawa', 'barat', 'jawa', 'jawa', 'timur', 'kalimantan', 'barat', 'kalimantan', 'timur', 'sulawesi', 'sela

In [5]:
# Membuat kamus: memetakan setiap kata unik ke sebuah ID
dictionary = corpora.Dictionary(processed_docs)

# Opsional: Filter kata-kata yang terlalu jarang atau terlalu sering muncul
dictionary.filter_extremes(no_below=2, no_above=0.7)
print(f"Dictionary berhasil dibuat dengan {len(dictionary)} kata unik setelah filtering.")

# Membuat corpus: mengubah setiap dokumen menjadi representasi BoW (ID kata, frekuensi)
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
print("\nCorpus berhasil dibuat.")
print("Contoh representasi BoW untuk dokumen pertama:")
print(corpus[0])

Dictionary berhasil dibuat dengan 179 kata unik setelah filtering.

Corpus berhasil dibuat.
Contoh representasi BoW untuk dokumen pertama:
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1)]


## LDA model

In [6]:
# Tentukan jumlah topik yang ingin dicari
num_topics = 5

print(f"Memulai pelatihan model LDA untuk menemukan {num_topics} topik...")
if not corpus or not dictionary:
    print("Corpus atau Dictionary kosong. Tidak dapat melatih model LDA.")
else:
    lda_model = models.LdaModel(
        corpus=corpus,
        id2word=dictionary,
        num_topics=num_topics,
        random_state=100,  # Agar hasil bisa direproduksi
        passes=15          # Jumlah iterasi pelatihan
    )
    print("Model LDA berhasil dilatih!")

Memulai pelatihan model LDA untuk menemukan 5 topik...
Model LDA berhasil dilatih!


In [7]:
if 'lda_model' in locals():
    print("Topik-topik yang ditemukan oleh model LDA:")
    print("-" * 50)
    for idx, topic in lda_model.print_topics(-1):
        print(f"Topik: {idx} \nKata Kunci: {topic}\n")

    # Evaluasi dengan Coherence Score dengan c_v
    coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print("-" * 50)
    print(f'Coherence Score (c_v): {coherence_lda:.4f}')
    print("(Skor Coherence yang baik biasanya mendekati 1.0)")
else:
    print("Model LDA belum dilatih. Jalankan sel sebelumnya terlebih dahulu.")

Topik-topik yang ditemukan oleh model LDA:
--------------------------------------------------
Topik: 0 
Kata Kunci: 0.075*"surabaya" + 0.055*"wib" + 0.040*"com" + 0.027*"korban" + 0.023*"ponpes" + 0.021*"lumajang" + 0.021*"sangka" + 0.020*"pamekasan" + 0.020*"kompas" + 0.017*"berita"

Topik: 1 
Kata Kunci: 0.037*"artikel" + 0.036*"news" + 0.036*"properti" + 0.030*"kompas" + 0.030*"jawa" + 0.030*"travel" + 0.029*"indonesia" + 0.023*"timur" + 0.023*"foto" + 0.023*"health"

Topik: 2 
Kata Kunci: 0.063*"ponpes" + 0.055*"al" + 0.051*"khoziny" + 0.039*"korban" + 0.038*"identifikasi" + 0.035*"jenazah" + 0.026*"keluarga" + 0.024*"santri" + 0.021*"tewas" + 0.020*"bangkal"

Topik: 3 
Kata Kunci: 0.056*"body" + 0.052*"jenazah" + 0.047*"part" + 0.046*"dvi" + 0.044*"kantong" + 0.029*"dna" + 0.029*"teliti" + 0.029*"rs" + 0.029*"bhayangkara" + 0.028*"pusdokkes"

Topik: 4 
Kata Kunci: 0.040*"identifikasi" + 0.036*"didik" + 0.027*"sehat" + 0.027*"khusnan" + 0.027*"dr" + 0.025*"baca" + 0.022*"umkm" + 0.